<a href="https://colab.research.google.com/github/JSRV18/Dinosaurio-repository/blob/main/LinealDinosaurio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
#Punto 2

import numpy as np

G0=(lambda x,y,z: 3*x - y - z - 1., \
    lambda x,y,z: -x + 3*y + z - 3., \
    lambda x,y,z: 2*x + y + 4*z - 7.)

A=np.array([[3,-1,-1],[-1,3,1],[2,1,4]])
b=np.array([1,3,7])

def gauss_seidel(A, b, x0, tol=1e-6, max_iter=100000):

   x = x0.copy()

   for it in range(max_iter):

    x_n = np.copy(x)

    for i in range(A.shape[0]):
      Suma = 0.
      for j in range(A.shape[1]):
        if i != j:
          Suma += A[i,j]*x_n[j]

          x_n[i] = (b[i] - Suma)/A[i,i]

    print(x_n)

    if np.linalg.norm((A@x_n)-b) < tol:
      break

    x = np.copy(x_n)


   return x_n, it



In [64]:
gauss_seidel(A,b,np.array([10,37,58]))

[ 32  -7 -12]
[-6  3  4]
[2 0 0]
[0 1 1]
[1 1 1]


(array([1, 1, 1]), 4)

In [22]:
np.linalg.solve(A,b)

array([1., 1., 1.])

In [14]:
# Punto 4



matriz1=np.array([[1,0,0],[5,1,0],[-2,3,1]])
matriz2=np.array([[4,-2,1],[0,3,7],[0,0,2]])


def multiplicar_matrices(matriz1, matriz2):
  n=len(matriz1)
  m=len(matriz2[0])
  p=len(matriz2)
  x=np.zeros((n,m))
  if n!=m:
    print("No se puede multiplicar")
  else:
    for i in range(n):
      for j in range(m):
        for k in range(p):
         x[i][j] += matriz1[i][k] * matriz2[k][j]
  return x

In [3]:
multiplicar_matrices(matriz1, matriz2)

array([[ 4., -2.,  1.],
       [20., -7., 12.],
       [-8., 13., 21.]])

In [4]:
# Punto 8

matriz=np.array([[4,-2,1],[20,-7,12],[-8,13,17]])

def LU(matriz):
  L=np.zeros_like(matriz)
  for i in range(len(L)):
    for j in range(len(L[0])):
      if i==j:
        L[i][j]=1

  U = matriz.astype(np.float64)
  n = U.shape[0]

  for k in range(n):
      for w in range(k+1,n):
          u = U[w,k]/U[k,k]
          L[w,k]=u
          U[w,:] -= u*U[k,:]

  return L,U

In [5]:
LU(matriz)

(array([[ 1,  0,  0],
        [ 5,  1,  0],
        [-2,  3,  1]]),
 array([[ 4., -2.,  1.],
        [ 0.,  3.,  7.],
        [ 0.,  0., -2.]]))

In [6]:
#Punto 12

F1=(lambda x,y: np.log(x**2+y**2) - np.sin(x*y) - np.log(2*np.pi), \
    lambda x,y: np.exp(x-y) + np.cos(x*y))

F2=(lambda x,y,z: 6*x - 2*np.cos(y*z) - 1, \
    lambda x,y,z: 9*y + np.sqrt(x**2 + np.sin(z) + 1.06) + 0.9, \
    lambda x,y,z: 60*z + 3*np.exp(-x*y) + 10*np.pi - 3)

def GetJacobian1(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1]) - f[i](rb[0],rb[1]))/(2*h)

  return J

def GetJacobian2(f,r,h=0.0001):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2]))/(2*h)

  return J


In [7]:
Ja1 = GetJacobian1(F1,np.array([2,2]))
Ja2 = GetJacobian2(F2,np.array([0.5,0.5,0.5]))


In [8]:
Ja2

array([[ 6.        ,  0.24740396,  0.24740396],
       [ 0.37377753,  9.        ,  0.32802064],
       [-1.16820118, -1.16820118, 60.        ]])

In [9]:
def newton1(F1, Ja, x0, tol=1e-6, max_iter=10000):

  error=1
  it=0
  while error>tol and it<max_iter:

    F1_vals = np.array([f(x0[0], x0[1]) for f in F1])
    x1=x0 - np.linalg.inv(Ja)@F1_vals
    error=np.linalg.norm(x1-x0)
    x0=x1
    it+=1
  return x0,it


def newton2(F2, x0, tol=1e-6, max_iter=10000):

  error=1
  it=0
  while error>tol and it<max_iter:
    Ja2 = GetJacobian2(F2,x0)

    F2_vals = np.array([f(x0[0], x0[1], x0[2]) for f in F2])
    if np.linalg.det(Ja2) == 0:
      from numpy.linalg import LinAlgError
      raise LinAlgError("Singular matrix encountered. Try a different initial guess.")
    x1=x0 - np.linalg.inv(Ja2)@F2_vals
    error=np.linalg.norm(x1-x0)
    x0=x1
    it+=1
  return x0,it

In [10]:
newton2(F2, np.array([0.1,0.1,0.1]))

(array([ 0.49814468, -0.1996059 , -0.52882598]), 4)

In [11]:
# Punto 13

def Jacobian_4(f,r,h=0.01):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      d_rf = r.copy()
      d_rb = r.copy()
      rf = r.copy()
      rb = r.copy()

      d_rf[j] = d_rf[j] + 2*h
      rf[j] = rf[j] + h
      rb[j] = rb[j] - h
      d_rb[j] = d_rb[j] - 2*h

      J[i,j] = (-f[i](d_rf[0],d_rf[1],d_rf[2]) + 8*f[i](rf[0],rf[1],rf[2]) - 8*f[i](rb[0],rb[1],rb[2]) + f[i](d_rb[0],d_rb[1],d_rb[2]))/(12*h)

  return J

In [12]:
Jacobian_4(F2,np.array([0.5,0.5,0.5]))

array([[ 6.        ,  0.24740396,  0.24740396],
       [ 0.37377753,  9.        ,  0.32802064],
       [-1.16820117, -1.16820117, 60.        ]])

In [13]:
GetJacobian2(F2,np.array([0.5,0.5,0.5]))

array([[ 6.        ,  0.24740396,  0.24740396],
       [ 0.37377753,  9.        ,  0.32802064],
       [-1.16820118, -1.16820118, 60.        ]])